In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tables
import os

In [10]:
a = np.array([2, 3, 7, 4, 8, 5, 3, 6, 8, 9, 5, 3, 1])
b = np.array([4, 6, 8, 2, 2, 4, 4, 3, 5, 1, 3, 4, 5])
print(np.intersect1d(a, b, return_indices = True))

(array([1, 2, 3, 4, 5, 6, 8]), array([12,  0,  1,  3,  5,  7,  4], dtype=int64), array([9, 3, 7, 0, 8, 1, 2], dtype=int64))


In [15]:
lis = [(2, 3), (4, 7), (5, 2), (4, 9)]
for i in range(4):
    for j in range(4):
        print((i, j) in lis)

False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False


In [2]:
f = h5py.File("events.h5", "r")
print(f.filename, ":")

events.h5 :


In [3]:
rectify_map = h5py.File("rectify_map.h5", "r")
print(rectify_map.filename, ":")

rectify_map.h5 :


In [4]:
print([key for key in f.keys()], "\n") 
print([key for key in rectify_map.keys()], "\n") 

['events', 'ms_to_idx', 't_offset'] 

['rectify_map'] 



In [5]:
offset = int(np.array(f['t_offset']))
offset

49599200165

In [6]:
print([key for key in f['events'].keys()], "\n")

['p', 't', 'x', 'y'] 



In [7]:
print(np.array(f['ms_to_idx'])[0:20])

[     0  10478  21074  31409  41793  52263  62698  73216  83686  94157
 104559 115218 125728 136085 146461 156861 167397 177682 188020 198376]


In [8]:
f['ms_to_idx'].shape

(11902,)

In [9]:
t = f['events']['t']
t = np.array(t)
print(t.min(), t.max())

0 11900999


In [10]:
x = f['events']['x']
x = np.array(x)
print(x.min(), x.max())

0 639


In [11]:
y = f['events']['y']
y = np.array(y)
print(y.min(), y.max())

0 479


In [12]:
t = f['events']['t']
t = np.array(t)
t = t + offset
print(t.min(), t.max())
print(np.argwhere(t < 49599400524))

49599200165 49611101164
[[      0]
 [      1]
 [      2]
 ...
 [1800365]
 [1800366]
 [1800367]]


In [13]:
p = f['events']['p']
p = np.array(p)
print(p.min(), p.max())

0 1


In [14]:
event_data = np.vstack((x, y))
event_data = np.vstack((event_data, t))
event_data = np.vstack((event_data, p))
event_data = event_data.T
event_data
print(len(event_data))
event_data = event_data.astype(np.int64)
event_data
#x, y, t, p

134133270


array([[        622,         279, 49599200165,           1],
       [        126,         278, 49599200165,           1],
       [        534,         278, 49599200165,           0],
       ...,
       [        565,         316, 49611101164,           1],
       [        579,         316, 49611101164,           0],
       [        627,         316, 49611101164,           0]], dtype=int64)

In [15]:
rectify = rectify_map['rectify_map']
print(rectify.shape)
rectify[235, 577, 1]

(480, 640, 2)


240.43445

In [16]:
# for row in range(len(event_data)):
#     rectify_x = rectify[event_data[row, 1], event_data[row, 0], 0]
#     rectify_y = rectify[event_data[row, 1], event_data[row, 0], 1]
#     event_data[row, 0] = rectify_x
#     event_data[row, 1] = rectify_y
# event_data


event_data[:, 0:2] = rectify[:][event_data[:, 1], event_data[:, 0], :].astype(int)
event_data

array([[        632,         286, 49599200165,           1],
       [        118,         283, 49599200165,           1],
       [        540,         284, 49599200165,           0],
       ...,
       [        572,         324, 49611101164,           1],
       [        587,         324, 49611101164,           0],
       [        637,         324, 49611101164,           0]], dtype=int64)

In [17]:
np.save('original_events', event_data)

In [ ]:
path = 'disparity/event'
files= os.listdir(path)
len(files)

In [ ]:
time_stamps = []
with open('disparity/timestamps.txt') as f:
    flag = 0
    for line in f.readlines():
        line = line.replace('\n', '')
        line  = int(line)
        time_stamps.append(line)
time_stamps = np.array(time_stamps)
len(time_stamps)

In [ ]:
index = np.zeros((len(event_data), 1))
event_data = np.hstack((event_data, index))

In [ ]:
event_data
#x, y, t, p, disp_index

In [ ]:
print(event_data[:, 2].min())
print(event_data[:, 2].max())

In [ ]:
for row in range(len(event_data)):
    event_data[row, 4] = np.argmin(np.abs(time_stamps - event_data[row, 2]))
event_data

In [ ]:
index = np.zeros((len(event_data), 1))
event_data = np.hstack((event_data, index))
event_data

In [ ]:
print(len(event_data))
out_of_range_index = np.argwhere(np.logical_or(np.logical_or(event_data[:, 0] > 639, event_data[:, 0] < 0),
                                 np.logical_or(event_data[:, 1] > 479, event_data[:, 0] < 0)))
print(len(out_of_range_index))
event_data = np.delete(event_data, out_of_range_index, axis = 0)
print(len(event_data))

In [ ]:
last_disparity_file = 0
for row in range(len(event_data)):
    current_disparity_file = files[event_data[row, 4].astype(int)]
    #print(disparity_file)
    if current_disparity_file != last_disparity_file:
        print("Different!")
        print(current_disparity_file)
        disparity_map = cv2.imread('disparity/event/' + current_disparity_file, 0)
    #plt.imshow(disparity_map)
    event_data[row][5] = disparity_map[event_data[row, 1].astype(int), event_data[row, 0].astype(int)]
    last_disparity_file = current_disparity_file
event_data

In [ ]:
print(len(event_data))
zero_index = np.argwhere(event_data[:, 5] == 0)
print(zero_index.shape)

In [ ]:
event_data = np.delete(event_data, zero_index, axis = 0)
event_data

In [ ]:
len(event_data)

In [ ]:
reprojection_matrix = np.array([[1.0, 0.0, 0.0, -335.0999870300293],
                               [0.0, 1.0, 0.0, -221.23667526245117],
                               [0.0, 0.0, 0.0, 569.7632987676102],
                               [0.0, 0.0, 1.6691407297504248, -0.0]])
R_rect0 = np.array([[0.9999313912417018, -0.0023139054373197965, 0.011482972222461762],
                    [0.002353841678837691, 0.9999912245858043, -0.003465570766066675],
                    [-0.011474852451585301, 0.0034923620961737592, 0.9999280629966356]])

In [ ]:
np.vstack((event_data[:, [0, 1, 5]].T, np.ones((1, len(event_data)))))

In [ ]:
event_3d = np.dot(reprojection_matrix, np.vstack((event_data[:, [0, 1, 5]].T, np.ones((1, len(event_data))))))
event_3d

In [ ]:
event_3d = event_3d/event_3d[3, :]
event_3d

In [ ]:
event_data = np.hstack((event_data, event_3d[0:3, :].T))
event_data

In [ ]:
T_event_to_frame = np.array([[0.9996874046885865, 0.009652146488870916, 0.023063585478994113, -0.04410263392688484],
                             [-0.009722042371104245, 0.9999484753460813, 0.0029203673010648615, 0.0005281285423087664],
                             [-0.023034209322743096, -0.0031436795631953228, 0.9997297347181744, -0.01229891454144492],
                             [0.0, 0.0, 0.0, 1.0]])

In [ ]:
event_3d_original = np.dot(np.linalg.inv(R_rect0), event_3d[0:3, :])
event_3d_original

In [ ]:
event_3d = np.vstack((event_3d_original, np.ones((1, len(event_data)))))
event_3d

In [ ]:
frame_3d = np.dot(T_event_to_frame, event_3d)
frame_3d

In [ ]:
event_data[:, [6, 7, 8]] = frame_3d[0:3, :].T
event_data

In [ ]:
print(frame_3d[0, :].max(), frame_3d[0, :].min())
print(frame_3d[1, :].max(), frame_3d[1, :].min())
print(frame_3d[2, :].max(), frame_3d[2, :].min())

In [ ]:
#event_data format right now: event_x, event_y, t, disparity_index, disparity, frame_x, frame_y, frame_z
R_rect1 = np.array([[0.9998572179847892, -0.013025778024398856, -0.010764420587133948],
                    [0.013060715513432202, 0.9999096430275752, 0.003181743349841093],
                    [0.01072200326407413, -0.0033218800890692088, 0.9999369998948329]])
K_rect1 = np.array([[1164.6238115833075, 0.0, 713.5791168212891],
                    [0.0, 1164.6238115833075, 570.9349365234375],
                    [0.0, 0.0, 1.0]])

In [ ]:
rectframe_3d = np.dot(R_rect1, frame_3d[0:3, :])
rectframe_3d

In [ ]:
rectframe = np.dot(K_rect1, rectframe_3d)
rectframe

In [ ]:
rectframe = rectframe/rectframe[2, :]
rectframe

In [ ]:
rectframe = rectframe[0:2, :]
rectframe

In [ ]:
print(rectframe[0, :].min(), rectframe[0, :].max())
print(rectframe[1, :].min(), rectframe[1, :].max())

In [ ]:
event_data[:, [6, 7, 8]] = rectframe_3d.T
event_data

In [ ]:
event_data = np.hstack((event_data, rectframe.T))
print(event_data.shape)
event_data[0:3]
#event_data format right now: event_x, event_y, t, p, disparity_index, disparity, rect3d_x, rect3d_y, rect3d_z,
#rectframe_x, rectframe_y

In [ ]:
frame_time_stamps = []
with open('dtimestamps.txt') as f:
    flag = 0
    for line in f.readlines():
        line = line.replace('\n', '')
        line  = int(line)
        frame_time_stamps.append(line)
frame_time_stamps = np.array(frame_time_stamps)
len(frame_time_stamps)

In [ ]:
#event_data format right now: event_x, event_y, t, disparity_index, disparity, rect3d_x, rect3d_y, rect3d_z,
#rectframe_x, rectframe_y, frame_timestamps_idx
event_data = np.hstack((event_data, np.zeros((len(event_data), 1))))
event_data.shape

In [ ]:
for row in range(len(event_data)):
    event_data[row, 11] = np.argmin(np.abs(frame_time_stamps - event_data[row, 2])) * 1000
event_data

In [ ]:
# image_id = -6
# with open('images.txt') as f:
#     for line in f.readlines():
#         image_id += 1
#         temp = line.strip().split()
#         if (temp[0] != '#') & (image_id %2 == 0 ) & (image_id/2 < 5) & (image_id/2 >= 0):
#             yes = 0
#             for row in range(len(event_data)):
#                 if event_data[row, 10] == image_id/2:
#                     for idx in range(int(len(temp)/3)):
#                         if temp[idx*3+2] != '-1':
#                             if (np.abs(float(temp[3*idx])-event_data[row, 8])<5) & (np.abs(float(temp[3*idx+1])-event_data[row, 9]) <5):
#                                 yes += 1
#             print(yes)

In [ ]:
# image_id = -6
# with open('images.txt') as f:
#     for line in f.readlines():
#         image_id += 1
#         temp = line.strip().split()
#         if (temp[0] != '#') & (image_id %2 == 0 ) & (image_id/2 < 5) & (image_id/2 >= 0):
#             for row in range(len(event_data)):
#                 if event_data[row, 10] == image_id/2:
#                     for idx in range(int(len(temp)/3)):
#                         if temp[idx*3+2] != '-1':
#                             if (np.abs(float(temp[3*idx])-event_data[row, 8])<5) & (np.abs(float(temp[3*idx+1])-event_data[row, 9]) <5):
#                                 event_data[row, 11] = float(temp[idx*3+2])
# np.save('event_data', event_data)

In [ ]:
#这个地方有没有什么更快的方法
flag = -4
compare_array = np.zeros((1, 4))
with open('images.txt') as f:
    for line in f.readlines():
        flag += 1
        if (line[0] != '#') & (flag % 2 == 0):
            temp = line.strip().split()
            for idx in range(int(len(temp)/3)):
                if temp[idx*3+2] != '-1':
                    line = np.zeros((1, 4))
                    line[0, 0] = float(temp[3*idx])
                    line[0, 1] = float(temp[3*idx + 1])
                    line[0, 2] = flag/2 * 1000
                    line[0, 3] = float(temp[3*idx + 2])
                    compare_array = np.vstack((compare_array, line))
compare_array

In [ ]:
compare_array = np.delete(compare_array, 0, axis = 0)
compare_array

In [ ]:
compare_array[:, 2] = compare_array[:, 2] - 1000
compare_array

In [ ]:
np.save( 'compare_array.npy', compare_array)

In [ ]:
compare_array = np.load('compare_array.npy')
# event_data = np.load('event_data.npy')

In [ ]:
compare_array

In [ ]:
event_data

In [ ]:
from scipy import spatial
print(compare_array[:, 0:3].shape)
print(event_data[:, (9, 10, 11)].shape)
kd_tree = spatial.KDTree(compare_array[:, 0:3])
dist, idx = kd_tree.query(event_data[:, (9, 10, 11)])

In [ ]:
print(dist.max())
print(dist.min())
print(dist.shape)
idx_idx = np.argwhere(dist<3)
print(idx_idx.shape)

In [ ]:
print(idx[idx_idx].shape)

In [ ]:
event_data = event_data[idx_idx, :]

In [ ]:
event_data = event_data.squeeze()
event_data.shape

In [ ]:
compare_array[idx[idx_idx]].squeeze()

In [ ]:
event_data = np.hstack((event_data, compare_array[idx[idx_idx]].squeeze()[:, 3].reshape(-1, 1)))
event_data.shape

In [ ]:
event_data[2, 12].dtype

In [ ]:
event_data[:, 12]

In [ ]:
np.save( 'event_data.npy', event_data)

In [ ]:
effective_event_data = event_data
effective_event_data[-1,  :]

In [ ]:
print(effective_event_data.shape)
time_stamps = effective_event_data[:, 11]
time_stamps = time_stamps.squeeze()
print(time_stamps.shape)
for i in np.unique(time_stamps):
    idx = np.argwhere(time_stamps == i)
    print('index for {}:\n'.format(i), idx.reshape(1, -1).squeeze(0))

In [ ]:
img0_index = np.argwhere(effective_event_data[:, 10] == 0)
img0 = effective_event_data[img0_index, :]
img0 = img0.squeeze()
img0

In [ ]:
img1_index = np.argwhere(effective_event_data[:, 10] == 1000)
img1 = effective_event_data[img1_index, :]
img1 = img1.squeeze()
img1

In [ ]:
img2_index = np.argwhere(effective_event_data[:, 10] == 2000)
img2 = effective_event_data[img2_index, :]
img2 = img2.squeeze()
img2

In [ ]:
img3_index = np.argwhere(effective_event_data[:, 10] == 3000)
img3 = effective_event_data[img3_index, :]
img3 = img3.squeeze()
img3

In [ ]:
img4_index = np.argwhere(effective_event_data[:, 10] == 4000)
img4 = effective_event_data[img4_index, :]
img4 = img4.squeeze()
img4

In [ ]:
image0 = np.ones([480, 640])
xx_img0 = img0[:, 0].astype(int)
yy_img0 = img0[:, 1].astype(int)
image0[yy_img0, xx_img0] = 2

image1 = np.ones([480, 640])
xx_img1 = img1[:, 0].astype(int)
yy_img1 = img1[:, 1].astype(int)
image1[yy_img1, xx_img1] = 2

image2 = np.ones([480, 640])
xx_img2 = img2[:, 0].astype(int)
yy_img2 = img2[:, 1].astype(int)
image2[yy_img2, xx_img2] = 2

image3 = np.ones([480, 640])
xx_img3 = img3[:, 0].astype(int)
yy_img3 = img3[:, 1].astype(int)
image3[yy_img3, xx_img3] = 2

image4 = np.ones([480, 640])
xx_img4 = img4[:, 0].astype(int)
yy_img4 = img4[:, 1].astype(int)
image4[yy_img4, xx_img4] = 2



plt.figure(1, (15, 10))
plt.subplot(1, 2, 1)
plt.imshow(image0)
plt.subplot(1, 2, 2)
plt.imshow(image1)
plt.show()

In [ ]:
plt.figure(1, (15, 10))
plt.subplot(1, 2, 1)
plt.imshow(image2)
plt.subplot(1, 2, 2)
plt.imshow(image3)
plt.show()

In [ ]:
check_image = np.ones([480, 640*2])
xx_check_image = img1[:, 0].astype(int)
xx_check_image.shape

In [ ]:
whole_events = np.vstack((x[0:1800367], y[0:1800367]))
whole_events = np.vstack((whole_events, t[0:1800367]))
whole_events = whole_events.T
print(whole_events.shape)
index_for_img3 = np.argmin(np.abs(frame_time_stamps[2] - event_data[:, 2]))
index_for_img3

In [ ]:
check_image = np.zeros([480, 640*2])
plt.figure(1, (15, 10))

# xx_img3 = img3[:, 0].astype(int)
# yy_img3 = img3[:, 1].astype(int)
# check_image[yy_img3, xx_img3] = 2
# xx_img4 = img4[:, 0].astype(int) + 640
# yy_img4 = img4[:, 1].astype(int)
# check_image[yy_img4, xx_img4] = 2

for row in range(whole_events.shape[0]):
    index_for_events = np.argmin(np.abs(frame_time_stamps - whole_events[row, 2]))
    if (index_for_events == 2):
        if (whole_events[row, 0]<640) & (whole_events[row, 1]<480):
            check_image[whole_events[row, 1].astype(int), whole_events[row, 0].astype(int)] = 2
    if (index_for_events == 3):
        if (whole_events[row, 0]<640) & (whole_events[row, 1]<480):
            check_image[whole_events[row, 1].astype(int), whole_events[row, 0].astype(int) + 640] = 2
    
print(effective_event_data.shape)
time_stamps = effective_event_data[:, 11]
time_stamps = time_stamps.squeeze()
index = effective_event_data[:, 10].squeeze()
print(time_stamps)
print(index)
k = 0
for i in np.unique(time_stamps):
    idx_1 = np.argwhere(np.logical_and(time_stamps == i, index == 3000))
    idx_2 = np.argwhere(np.logical_and(time_stamps == i, index == 4000))
    if (idx_1.shape[0] != 0) & (idx_2.shape[0] != 0):
        point_1 = idx_1.reshape(1, -1).squeeze(0)[0]
        point_2 = idx_2.reshape(1, -1).squeeze(0)[0]
#         print('index for {}:\n'.format(i), point_1)
#         print('index for {}:\n'.format(i), point_2)
        x = [effective_event_data[point_1, 0], effective_event_data[point_2, 0] + 640]
        y = [effective_event_data[point_1, 1], effective_event_data[point_2, 1]]
        check_image[effective_event_data[point_1, 1].astype(int), effective_event_data[point_1, 0].astype(int)] = 1
        check_image[effective_event_data[point_2, 1].astype(int), effective_event_data[point_2, 0].astype(int) + 640] = 1
        plt.plot(x, y, color="white", linewidth=0.2)
plt.imshow(check_image)
plt.show()

In [ ]:
original_event_data[-1, :]

In [ ]:
time_stamps = effective_event_data[:, 11]
time_stamps = time_stamps.squeeze()
index = effective_event_data[:, 10].squeeze()
for frame_index in range(45):
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
    check_image = np.zeros((480, 640))
    for row in range(original_event_data.shape[0]):
        index_for_events = np.argmin(np.abs(frame_time_stamps - original_event_data[row, 2]))
        if (index_for_events == frame_index):
            if (original_event_data[row, 0]<640) & (original_event_data[row, 1]<480):
                check_image[original_event_data[row, 1].astype(int), original_event_data[row, 0].astype(int)] = 2   
    plt.imshow(check_image)
    for i in np.unique(time_stamps):
        idx_1 = np.argwhere(np.logical_and(time_stamps == i, index == 1000 * frame_index))
        idx_2 = np.argwhere(np.logical_and(time_stamps == i, index == 1000 * (frame_index+1)))
        if (idx_1.shape[0] != 0) & (idx_2.shape[0] != 0):
#             print(idx_1.shape)
#             print(idx_2.shape)
            point_1 = idx_1.reshape(1, -1).squeeze(0)[0]
            point_2 = idx_2.reshape(1, -1).squeeze(0)[0]
    #         print('index for {}:\n'.format(i), point_1)
    #         print('index for {}:\n'.format(i), point_2)
            X = effective_event_data[point_1, 0].astype(int)
            Y = effective_event_data[point_1, 1].astype(int)
            U = effective_event_data[point_2, 0] - effective_event_data[point_1, 0]
            V = effective_event_data[point_2, 1] - effective_event_data[point_1, 1]
            plt.quiver(X, Y, U, V, angles='xy', scale_units= 'xy', scale=1)
    plt.savefig('animation/' + str(frame_index) + '.png')

In [ ]:
import cv2
size = (432,288)
print(size)
#完成写入对象的创建，第一个参数是合成之后的视频的名称，第二个参数是可以使用的编码器，第三个参数是帧率即每秒钟展示多少张图片，第四个参数是图片大小信息
videowrite = cv2.VideoWriter(r'test_background.mp4',-1,3,size)
img_array=[]
for filename in [r'animation/{0}.png'.format(i) for i in range(45)]:
    img = cv2.imread(filename)
    if img is None:
        print(filename + " is error!")
        continue
    img_array.append(img)
for i in range(45):
    videowrite.write(img_array[i])
print('end!')

In [ ]:
compare_array = np.load('compare_array.npy')
event_data = np.load('event_data.npy')
print(compare_array[:, 0:3].shape)
print(event_data[:, (8, 9, 10)].shape)

In [ ]:
from scipy import spatial
kd_tree = spatial.KDTree(compare_array[:, 0:3])
dist, idx = kd_tree.query(event_data[:, (8, 9, 10)])

In [ ]:
print(dist.max())
print(dist.min())
print(dist.shape)
idx_idx = np.argwhere(dist<3)
print(idx_idx.shape)

In [ ]:
compare_array

In [ ]:
idx.shape

In [ ]:
effective_compare_array = compare_array[idx[idx_idx]].squeeze()
effective_compare_array

In [ ]:
event_data_compare = event_data[idx_idx]
event_data_compare

In [ ]:
event_data_compare = event_data_compare.squeeze()
event_data_compare

In [ ]:
event_data_compare.shape

In [ ]:
event_data[0, :]

In [ ]:
event_data_compare = np.hstack((event_data_compare, compare_array[idx[idx_idx]].squeeze()[:, 3].reshape(-1, 1)))
event_data_compare.shape

In [ ]:
event_data_compare[3, :]

In [ ]:
print(event_data_compare.shape)
point_id = event_data_compare[:, 11]
point_id = point_id.squeeze()
print(point_id.shape)
for i in np.unique(point_id):
    idx = np.argwhere(point_id == i)
    print('index for {}:\n'.format(i), idx.reshape(1, -1).squeeze(0))

In [ ]:
original_event_data[0:20]

In [ ]:
for i in np.unique(point_id):
    idx = np.argwhere(point_id == i)
    if i == 61519:
        event_data_for_one_feature = event_data_compare[idx]
        event_data_for_one_feature = event_data_for_one_feature.squeeze()
        flag = -1
        for row in range(event_data_for_one_feature.shape[0]):
            if event_data_for_one_feature[row, 10] != flag:
                #print(event_data_for_one_feature[row, :])
                flag = event_data_for_one_feature[row, 10]
                #print(dist_for_one_feature_and_frame)
                idx = np.where((np.abs(original_event_data[:, 0] - event_data_for_one_feature[row, 0]) < 5) & (np.abs(original_event_data[:, 1] - event_data_for_one_feature[row, 1]) < 5) & (np.abs(original_event_data[:, 2] - event_data_for_one_feature[row, 2]) < 10000))
                x_values = original_event_data[idx, 0]
                y_values = original_event_data[idx, 1]
                t_values = original_event_data[idx, 2]
#                 plt.xlim((event_data_for_one_feature[row, 0] - 10, event_data_for_one_feature[row, 0] + 10))
#                 plt.ylim((event_data_for_one_feature[row, 1] - 10, event_data_for_one_feature[row, 1] + 10))
#                 plt.scatter(x_values, y_values, s=5)
#                 plt.show()
#                 plt.clf()
                plt.xlim((event_data_for_one_feature[row, 0] - 10, event_data_for_one_feature[row, 0] + 10))
                plt.ylim((event_data_for_one_feature[row, 2] - 15000, event_data_for_one_feature[row, 2] + 15000))
                plt.scatter(x_values, t_values, s=5)
                plt.show()
                plt.clf()
#                 plt.scatter(y_values, t_values, s=5)
#                 plt.show()
#                 plt.clf()

In [ ]:
print(original_event_data.shape)
idx = np.where((np.abs(original_event_data[:, 0] - event_data_for_one_feature[120, 0]) < 5) & (np.abs(original_event_data[:, 1] - event_data_for_one_feature[120, 1]) < 5) & (np.abs(original_event_data[:, 2] - event_data_for_one_feature[120, 2]) < 5000))
print(idx)
x_values = original_event_data[idx, 0]
y_values = original_event_data[idx, 1]
t_values = original_event_data[idx, 2]
plt.scatter(x_values, y_values, s=50)
plt.show()